In [1]:
import pandas as pd

df17 = pd.read_excel('2017matchdataOraclesElixir.xlsx')
df18_1 = pd.read_excel('2018 summer match data OraclesElixir 2018-09-27.xlsx')
df18_2 = pd.read_excel('2018-spring-match-data-OraclesElixir-2018-05-20.xlsx')
df18_3 = pd.read_excel('2018-worlds-match-data-OraclesElixir-2018-11-03.xlsx')

df19_1 = pd.read_excel('2019-spring-match-data-OraclesElixir-2019-05-21.xlsx')
df19_2 = pd.read_excel('2019-summer-match-data-OraclesElixir-2019-09-16.xlsx')
df19_3 = pd.read_excel('2019-summer-match-data-OraclesElixir-2019-11-10.xlsx')

df = pd.concat([df17,df18_1,df18_2,df18_3,df19_1,df19_2],ignore_index=True)
df = df[df.league != 'LPL']
df.head()

,gameid,url,league,split,date,week,game,patchno,playerid,side,...,gdat15,xpat10,oppxpat10,xpdat10,csat10,oppcsat10,csdat10,csat15,oppcsat15,csdat15
0,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,1,Blue,...,123.0,4669.0,4677.0,-8.0,94.0,89.0,5.0,150.0,136.0,14.0
1,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,2,Blue,...,-756.0,3134.0,3161.0,-27.0,47.0,47.0,0.0,62.0,79.0,-17.0
2,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,3,Blue,...,-1050.0,5088.0,5169.0,-81.0,93.0,92.0,1.0,142.0,143.0,-1.0
3,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,4,Blue,...,-1057.0,3141.0,2984.0,157.0,68.0,75.0,-7.0,112.0,128.0,-16.0
4,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,5,Blue,...,-1114.0,2593.0,2816.0,-223.0,5.0,2.0,3.0,7.0,2.0,5.0


In [49]:
df = df.reset_index(drop=True)
def select_by_player(df, player):
    return df.loc[df['player'] == player]

def select_by_champ(df, champ):
    return df.loc[df['champion'] == champ]

def player_score(df, player):
    df_new = select_by_player(df, player)
    score = []
    for index, row in df_new.iterrows():
        s = ((row['minionkills']+row['monsterkills'])*0.5+row['k']*3+row['a']*2-row['d']*4+row['wards']*2)/row['gamelength']
        score.append(s)
        
    df_new['score'] = score
    return df_new['score'].mean()

def champ_score(df, player, champ):
    df_new = select_by_champ(df, champ)
    df2 = select_by_player(df_new, player)
    score = []
    for index, row in df2.iterrows():
        s = ((row['minionkills']+row['monsterkills'])*0.5+row['k']*3+row['a']*2-row['d']*4+row['wards']*2)/row['gamelength']
        score.append(s)
        
    df2['score'] = score
    return df2['score'].mean()

def champ_score_avg(df, champ):
    df_new = select_by_champ(df, champ)
    score = []
    for index, row in df_new.iterrows():
        s = ((row['minionkills']+row['monsterkills'])*0.5+row['k']*3+row['a']*2-row['d']*4+row['wards']*2)/row['gamelength']
        score.append(s)
        
    df_new['score'] = score
    return df_new['score'].mean()

In [50]:
players = df.player.unique()
# scores for all players
p_score = {}
for name in players:
    p_score[name]=player_score(df,name) 

champions = df.champion.unique()
# scores for all champions
c_score = {}
for name in champions:
    c_score[name]={}
    for p in players:
        c_score[name][p]=champ_score(df,p,name)

/Users/evelyn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/evelyn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [51]:
# scores for all champions average
c_score_avg = {}
for name in champions:
    c_score_avg[name]=champ_score_avg(df,name)

/Users/evelyn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [52]:
# Construct the train dataset
COLUMN_NAMES=['Blue_1','Blue_2','Blue_3','Blue_4','Blue_5','Red_1','Red_2','Red_3','Red_4','Red_5',
              'Blue_1_champ','Blue_2_champ','Blue_3_champ','Blue_4_champ','Blue_5_champ',
              'Red_1_champ','Red_2_champ','Red_3_champ','Red_4_champ','Red_5_champ',
              'Blue_Win']
train = pd.DataFrame(columns=COLUMN_NAMES)
row_cnt = df.shape[0]
row_cnt

74016

In [53]:
game_id = df.gameid.unique()
index=0
candidate_Score = []
champion_Score = []

In [54]:
import numpy as np
i = 0
index = 0
while i<row_cnt:
    if df.loc[i]['player']!= 'Team':
        name = df.loc[i]['player']
        candidate_Score.append(p_score[name])
    
        champ = df.loc[i]['champion']
        if np.isnan(c_score[champ][name]):
            champion_Score.append(c_score_avg[champ])
        else:
            champion_Score.append(c_score[champ][name])
    # if Blue team wins, 1; Red wins 0
    if (i+2)%12 == 0:
        #print(i)
        result = int(df.loc[i].result)
        new_row = candidate_Score + champion_Score
        new_row.append(result)
        train.loc[index]=new_row
        index += 1
        candidate_Score.clear()
        champion_Score.clear()
    i += 1

In [73]:
lol=pd.read_csv('lol_data.csv')
# 64% as training set, 16% as validation, 20% as test
X_train = lol.values[0:4021,0:20]
Y_train = lol.values[0:4021,20]
X_validate = lol.values[4021:5028,0:20]
Y_validate = lol.values[4021:5028,20]
X_test = lol.values[5028:,0:20]
Y_test = lol.values[5028:,20]

In [74]:
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

#X = lol.values[:,0:20]
#X = preprocessing.normalize(X)
#Y = lol.values[:,20]
#X_train, X_validate, Y_train, Y_validate = train_test_split(X, Y, test_size=0.2, random_state=0)
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, Y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, Y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_validate, Y_validate)))
    

Learning rate:  0.05
Accuracy score (training): 0.637
Accuracy score (validation): 0.594
Learning rate:  0.075
Accuracy score (training): 0.682
Accuracy score (validation): 0.630
Learning rate:  0.1
Accuracy score (training): 0.701
Accuracy score (validation): 0.656
Learning rate:  0.25
Accuracy score (training): 0.727
Accuracy score (validation): 0.692
Learning rate:  0.5
Accuracy score (training): 0.734
Accuracy score (validation): 0.703
Learning rate:  0.75
Accuracy score (training): 0.739
Accuracy score (validation): 0.685
Learning rate:  1
Accuracy score (training): 0.739
Accuracy score (validation): 0.696


In [76]:
gb_clf2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=0)
gb_clf2.fit(X_train, Y_train)
predictions = gb_clf2.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(Y_test, predictions))

print("Classification Report")
print(classification_report(Y_test, predictions))

Confusion Matrix:
[[359 203]
 [164 530]]
Classification Report
              precision    recall  f1-score   support

         0.0       0.69      0.64      0.66       562
         1.0       0.72      0.76      0.74       694

   micro avg       0.71      0.71      0.71      1256
   macro avg       0.70      0.70      0.70      1256
weighted avg       0.71      0.71      0.71      1256



In [57]:
player_name = []
print("Please input the name of Blue team player:")
x=input()
player_name=x.split(',')

print("Please input the name of Red team player:")
x=input()
player_name=player_name + x.split(',')

champ_name = []
print("Please input the name of Blue team champions:")
x=input()
champ_name=x.split(',')

print("Please input the name of Red team champions:")
x=input()
champ_name=champ_name + x.split(',')

Please input the name of Blue team player:


KeyboardInterrupt: 

In [ ]:
from statistics import mean

player_score=[]
for name in player_name:
    if name not in p_score:
        player_score.append(mean(p_score[k] for k in p_score))
        #player_score.append(np.mean(p_score.values()))
    else:
        player_score.append(p_score[name])

champ_Score=[]
for i in range(10):
    if (player_name[i] not in champ_name[i]) or (np.isnan(c_score[champ_name[i]][player_name[i]])):
        champ_Score.append(c_score_avg[champ_name[i]]-1)
    else:
        champ_Score.append(c_score[champ_name[i]][player_name[i]])
final_score = player_score+champ_Score

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

x = ['Top','Jungle','Middle','ADC','Support']
y1 = player_score[0:5]
y2 = player_score[5:10]


plt.plot(x, y1, 'o', color='blue',label="Blue Team='{0}'".format('o'))
plt.plot(x, y2, 'd', color='red',label="Red Team='{0}'".format('d'))
plt.legend(numpoints=1)

plt.plot(x, y1, '-ok',color='blue')
plt.plot(x, y2, '-ok',color='red')
plt.title('Confidence Scores by Players\' Historical Record')

In [ ]:
fig, axes = plt.subplots(ncols=2, sharey=True)
z1 = champ_Score[0:5]
z2 = champ_Score[5:10]
axes[0].barh(x, z1, align='center', color='blue')
axes[1].barh(x, z2, align='center', color='red')
axes[0].invert_xaxis()
axes[0].set(title='                                               Confidence scores by Player using specific Champion')
plt.show()



In [ ]:
input_para = np.reshape(final_score,(1,-1))
predictions = gb_clf2.predict_proba(preprocessing.normalize(input_para))
predictions
print("The win rate of Blue team is:", round(predictions[0][1],3))

In [ ]:
lol=pd.read_csv('lol_data.csv')
lol.shape

In [66]:
# 64% as training set, 16% as validation, 20% as test
X_train = lol.values[0:4021,0:20]
Y_train = lol.values[0:4021,20]
X_validate = lol.values[4021:5028,0:20]
Y_validate = lol.values[4021:5028,20]
X_test = lol.values[5028:,0:20]
Y_test = lol.values[5028:,20]

In [81]:
# --- Import Librarires ----
import tensorflow as tf
from keras import Sequential
from keras.layers import Input, Dense
from keras.models import Model


# --- Setting up a ReLu Sequential Model ---
model = Sequential()
model.add(Dense(64,activation='relu', kernel_initializer = 'uniform', input_shape = (20,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# Train neural network
from time import time
start_time = time()
history_64 = model.fit(X_train, Y_train,
              batch_size=64, # Batch sizes - runtime scales ~linearly
              verbose = 1,
              epochs=50, 
              shuffle=False,
              validation_data=(X_validate, Y_validate))
time_to_complete = time()- start_time
print("Batch size 64: takes {} seconds to complete".format(time_to_complete))



Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_63 (Dense)             (None, 64)                1344      
_________________________________________________________________
dense_64 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_65 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_66 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_67 (Dense)             (None, 1)                 65        
Total params: 13,889
Trainable params: 13,889
Non-trainable params: 0
_________________________________________________________________
Train on 4021 samples, validate on 1007 samples
Epoch 1/50
4021/4021 [==============================] - 0s 123us/st

Epoch 50/50
4021/4021 [==============================] - 0s 20us/step - loss: 0.5583 - accuracy: 0.7083 - val_loss: 0.5459 - val_accuracy: 0.7428
Batch size 64: takes 5.838427782058716 seconds to complete


In [82]:
# --- Evaluation metrics ---
import numpy as np
# Evaluation metrics
score = model.evaluate(X_test, Y_test, batch_size=64)
predict_probabilities = model.predict(X_test, batch_size = 64)
predict_classes = model.predict_classes(X_test, batch_size = 64)

print(
"""
On the test set:
The mean validation loss is: {mean_test_loss},
The mean accuracy score is: {mean_test_accuracy}
""".format(mean_test_loss = score[0],
           mean_test_accuracy = score[1]))

1256/1256 [==============================] - 0s 16us/step

On the test set:
The mean validation loss is: 0.5373428699317252,
The mean accuracy score is: 0.7205414175987244

